# Mid-Term - Part 1
Compare the two algorithms we studied, K-NN and RLS with linear models, in terms of accuracy and training time


In [128]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from scipy.interpolate import griddata

############################################################
def linearRegrFunction(n, D, low_D, high_D, W, sigma_noise):
    X = np.zeros((n,D))
    for i in range(0, D):
        X[:,i] = np.random.uniform(low_D[i], high_D[i], size=n)

    gauss_noise = np.random.normal(0, sigma_noise, size=(n,1))

    Y = np.dot(X, W) + gauss_noise

    return X, Y

#####################
def flipLabels(Y, P):
    if P < 1 or P > 100:
        raise Exception('P should be between 1 and 100')

    indices_to_flip = np.random.choice(range(len(Y)), int(len(Y) * (P / 100)), replace=False)
    Y_noisy = Y.copy()
    Y_noisy[indices_to_flip] *= -1

    return Y_noisy

### KNN-Classify

In [129]:
def calcError(Ypred, Ytrue):
    return (np.count_nonzero(Ypred != Ytrue)) / len(Ytrue)

#########################
def euclidDistance(P1,P2):
    return np.linalg.norm(P1-P2,2)

#########################
def allDistances(X1, X2):
    D = np.zeros((X1.shape[0], X2.shape[0]))
    for idx1 in range(len(X1)):
        for idx2 in range(len(X2)):
            D[idx1,idx2] = euclidDistance(X1[idx1,:],X2[idx2,:])
    return D

##################################
def kNNClassify(Xtr, Ytr, k, Xte):
    n_train = Xtr.shape[0]
    n_test = Xte.shape[0]

    if any(np.abs(Ytr) != 1):
        raise Exception("The values of Ytr should be +1 or -1.")

    if k > n_train:
        print("k is greater than the number of points, setting k=n_train")
        k = n_train

    Ypred = np.zeros(n_test)

    dist = allDistances(Xte, Xtr)

    for idx in range(n_test):
        idx_dist = dist[idx]
        order_idx_dist = np.argsort(idx_dist)
        k_idx_dist = order_idx_dist[:k]
        Ypred[idx] = np.sign(np.mean(Ytr[k_idx_dist]))

    return Ypred

#####################################################
def KFoldCVkNN(Xtr, Ytr, num_folds, hyperparam_list):
    rnd_state = np.random.RandomState()
    hyperparam_list = np.array(hyperparam_list)
    num_k = len(k_list)

    n_tot = Xtr.shape[0]

    tr_errors = np.zeros((num_k, num_folds))
    val_errors = np.zeros((num_k, num_folds))

    rand_idx = rnd_state.choice(n_tot, size=n_tot, replace=False)
    split_idx = np.array_split(rand_idx, num_folds)

    for fold_idx in range(num_folds):
        val_mask = np.zeros(n_tot, dtype=bool)
        val_mask[split_idx[fold_idx]] = True
        x_train = Xtr[val_mask==False]
        y_train = Ytr[val_mask==False]
        x_val = Xtr[val_mask==True]
        y_val = Ytr[val_mask==True]

        for k_idx, current_k in enumerate(hyperparam_list):
            tr_errors[k_idx, fold_idx] = calcError(kNNClassify(x_train, y_train, current_k, x_train), y_train)
            val_errors[k_idx, fold_idx] = calcError(kNNClassify(x_train, y_train, current_k, x_val), y_val)

    tr_err_mean = np.mean(tr_errors, axis=1)
    tr_err_std = np.std(tr_errors, axis=1)

    val_err_mean = np.mean(val_errors, axis=1)
    val_err_std = np.std(val_errors, axis=1)

    best_k = k_list[np.argmin(val_err_mean)]
    best_k_idx = np.atleast_1d(k_list == best_k).nonzero()

    return best_k, best_k_idx, tr_err_mean, tr_err_std, val_err_mean, val_err_std

####################################
def separatingFkNN(Xtr, Ytr, k, Xte):
    Ypred = kNNClassify(Xtr=Xtr, Ytr=Ytr, k=k, Xte=Xte)

    x = Xtr[:, 0]
    y = Xtr[:, 1]
    xi = np.linspace(x.min(), x.max(), 500)
    yi = np.linspace(y.min(), y.max(), 500)
    zi = griddata((x, y), Ypred, (xi[None, :], yi[:, None]), method='linear')

    plt.subplots()
    CS = plt.contour(xi, yi, zi, 15, linewidths=2, colors='k', levels=[0])
    plt.scatter(x[Ytr==1], y[Ytr==1], c="#C59434", marker='o', s=50, zorder=10, alpha=0.8)
    plt.scatter(x[Ytr==-1], y[Ytr==-1], c="#092C48", marker='o', s=50, zorder=10, alpha=0.8)
    plt.xlim(x.min(), x.max())
    plt.ylim(x.min(), x.max())

### RLS

In [130]:
def regularizedLSTrain(Xtr, Ytr, lam):
    n = Xtr.shape[0]
    A = Xtr.transpose() @ Xtr + lam*n*np.eye(Xtr.shape[1])
    b = Xtr.transpose() @ Ytr

    lower_triangular = np.linalg.cholesky(A)

    y = scipy.linalg.solve_triangular(lower_triangular, b, lower=True)
    w = scipy.linalg.solve_triangular(lower_triangular.T, y, lower=False)

    return w

##############################
def regularizedLSTest(w, Xte):
    Ypred = Xte @ w
    return Ypred

### Function for data loading

In [131]:
def load_dataset(name):
    X, y = [], []
    with open("{}".format(name), 'r') as f:
        for line in f.readlines():
            splitted = line.split(",")
            X.append(splitted[:-1])
            y.append(splitted[-1])
    X, y = np.asarray(X, dtype=np.float32), np.asarray(y, dtype=np.float32)
    return X, y

### Dataset 1:
- $D = 10$
- $n = 400$
- Noise $= 10\%$

With K-NN

In [142]:
n = 400
D = 10
low_D, high_D = -10, 10
w = np.array(1.0).reshape(1, 1)
noise = 10

Xtr, Ytr = linearRegrFunction(n, D, low_D, high_D, w, noise)

_, ax = plt.subplots()
ax.set_title("Linear Regression data")
ax.set_xlabel("x")
ax.set_ylabel("y")

IndexError: list index out of range

With RLS

### Dataset 2:
- $D = 10$
- $n = 4000$
- Noise $= 10\%$

### Dataset 3:
- $D = 4000$
- $n = 400$
- Noise $= 10\%$

Instructions to load the test set and to
evaluate the model: